<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Labs/deepNN_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-1 : House Price Prediction using Deep-Learning

In [22]:
## lib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense

import requests

## Data Collection

In [2]:
## pulling dataset from my GitHub

In [5]:
# function on my gist: https://gist.github.com/bc3b75e60d321baca00102378122512d.git
def downloadCSV(fileURL, saveAs='downloaded.csv'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  csv_file = open(saveAs, 'wb')
  
  csv_file.write(fileURLContent)
  csv_file.close()

In [6]:
dataLoc = "https://raw.githubusercontent.com/ravi-prakash1907/Machine-Learning-for-Cyber-Security/main/Labs/dataset/housepricedata.csv?token=AJGAAOBNEAI7J3AXQ53GLM3AX5NOU"

downloadCSV(dataLoc,'housepricedata.csv')

In [7]:
df = pd.read_csv("housepricedata.csv")

## Data Understanding

In [8]:
df.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
0,8450,7,5,856,2,1,3,8,0,548,1
1,9600,6,8,1262,2,0,3,6,1,460,1
2,11250,7,5,920,2,1,3,6,1,608,1
3,9550,7,5,756,1,0,3,7,1,642,0
4,14260,8,5,1145,2,1,4,9,1,836,1


In [9]:
## looking at the shape
df.shape

(1460, 11)

In [11]:
## looking for the detailed description of the dataset
df.describe()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea,AboveMedianPrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,10516.828082,6.099315,5.575342,1057.429452,1.565068,0.382877,2.866438,6.517808,0.613014,472.980137,0.498630
std,9981.264932,1.382997,1.112799,438.705324,0.550916,0.502885,0.815778,1.625393,0.644666,213.804841,0.500169
min,1300.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000
25%,7553.500000,5.000000,5.000000,795.750000,1.000000,0.000000,2.000000,5.000000,0.000000,334.500000,0.000000
50%,9478.500000,6.000000,5.000000,991.500000,2.000000,0.000000,3.000000,6.000000,1.000000,480.000000,0.000000
75%,11601.500000,7.000000,6.000000,1298.250000,2.000000,1.000000,3.000000,7.000000,1.000000,576.000000,1.000000
max,215245.000000,10.000000,9.000000,6110.000000,3.000000,2.000000,8.000000,14.000000,3.000000,1418.000000,1.000000


**Hence, our aim is to predict the value of last column i.e. `AboveMedianPrice`**

## Data Preparation

In [41]:
## train - test split : 70:30
X_train, X_test, Y_train, Y_test = train_test_split(df[df.columns[:-1]], df['AboveMedianPrice'], test_size=0.4)

In [42]:
## splitting the test set further in : test & validation set
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [43]:
## here traing data has 70%, while validation and testing have 15%-15% of the whole dataset

### Train-Test-Validation Data

In [44]:
# features
display(X_train.head(3))
display(X_test.head(3))
display(X_valid.head(3))

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
620,8248,3,3,864,1,0,2,5,0,0
1420,11700,6,6,708,2,1,3,7,1,776
579,12150,5,5,1050,2,0,4,7,0,352


,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
812,8712,5,5,540,1,0,2,4,0,504
1110,8000,6,5,773,2,1,3,8,1,431
527,14948,9,5,1452,2,1,3,11,1,858


,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
172,5306,7,7,1064,2,0,2,5,1,504
1439,11584,7,6,539,2,1,3,6,1,550
809,8100,5,5,849,2,0,2,11,0,360


In [45]:
# labels
display(Y_train.head(3))
display(Y_test.head(3))
display(Y_valid.head(3))

620     0
1420    1
579     0
Name: AboveMedianPrice, dtype: int64

812     0
1110    1
527     1
Name: AboveMedianPrice, dtype: int64

172     1
1439    1
809     0
Name: AboveMedianPrice, dtype: int64

In [56]:
## normalization
min_max_scaler = preprocessing.MinMaxScaler()

X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)
X_valid = min_max_scaler.fit_transform(X_valid)

## Model Build

In [58]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(10,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [59]:
model.compile(optimizer='sgd',
                loss='binary_crossentropy',
                metrics=['accuracy', 'mse'])

In [48]:
history = model.fit(X_train, Y_train, epochs=5)

Epoch 1/5
28/28 [==============================] - 1s 1ms/step - loss: 4103262.8241 - accuracy: 0.4766 - mse: 0.3795
Epoch 2/5
28/28 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.4718 - mse: 0.2500
Epoch 3/5
28/28 [==============================] - 0s 1ms/step - loss: 0.6932 - accuracy: 0.5009 - mse: 0.2500
Epoch 4/5
28/28 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5248 - mse: 0.2499
Epoch 5/5
28/28 [==============================] - 0s 1ms/step - loss: 0.6930 - accuracy: 0.5268 - mse: 0.2499


In [60]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.',end='')


early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history = model.fit(X_train, Y_train, epochs=1000, verbose=0, validation_split=0.1,
                    callbacks = [early_stop, PrintDot()] )
  
hist = pd.DataFrame(history.history)
hist['epoc'] = history.epoch

rmse_final = np.sqrt(float(hist['val_mse'].tail(1)))
print()
print("Final RMSE on val: ",round(rmse_final,3))


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.....................................
Final RMSE on val:  0.252


### Evaluation

In [66]:
mse, _, _ = model.evaluate(X_test, Y_test)
rmse = np.sqrt(mse)
print("RMSE on test: ",round(rmse,3))

10/10 [==============================] - 0s 2ms/step - loss: 0.2557 - accuracy: 0.8801 - mse: 0.0796
RMSE on test:  0.506


**without  normalization, the accuracy was 48%**